In [1]:
import torch
from tqdm import tqdm
import pandas as pd
import trl 

tqdm.pandas()

import transformers
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
import os

from transformers import BitsAndBytesConfig, AutoTokenizer, HfArgumentParser, AutoConfig, AutoModelForSequenceClassification, AutoModelForCausalLM
from datasets import load_dataset
from trl import ModelConfig, RewardConfig, RewardTrainer, get_kbit_device_map

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler

import torch
import os
import time
from peft import (
    LoraConfig,
    TaskType,
)
import wandb
from dotenv import load_dotenv

from huggingface_hub import login


load_dotenv() 

/data/data/amenon-data/EthicalLLM/ethics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Loading the  models

In [2]:


model_name = "mistralai/Mistral-7B-Instruct-v0.2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}

# Use 4 bit quantization for the main model 
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)
config = AutoConfig.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# lora config
LORA_R = 8
LORA_ALPHA = 24
LORA_DROPOUT = 0.1
MAX_LENGTH = 300
batch_size = 16
DATA_SIZE = 10000

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=LORA_R, 
    lora_alpha=LORA_ALPHA, 
    lora_dropout=LORA_DROPOUT,
    bias='none'
)

## -------------------- LOADING THE MISTRAL MODEL AND TOKENIZER -----------
    
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    model_max_length=MAX_LENGTH,
    padding="max_length", 
    truncation=True)

tokenizer.pad_token = tokenizer.eos_token
    
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    quantization_config=quantization_config,
    device_map=device_map,
)

final_model_name = "amenon/EthcalLLM-PPO-final"
final_model = AutoModelForCausalLM.from_pretrained(
    final_model_name,
    config=config,
    quantization_config=quantization_config,
    device_map=device_map)

print("Model loaded!")

/data/data/amenon-data/EthicalLLM/ethics/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.44s/it]


Model loaded!


# Loading the reward models for the scores 

In [3]:
## -------------------- LOADING THE REWARD MODELS -------------------------------

reward_models = []
aspects = ['action', 'consequences', 'motivation']
reward_config = AutoConfig.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", num_labels=2)

for aspect in aspects:
    reward_model = AutoModelForSequenceClassification.from_pretrained(
        f"Tachi67/EthcalLLM-RM-{aspect}", 
        config=reward_config,
        quantization_config=quantization_config,
        device_map=device_map
    )
    reward_model.resize_token_embeddings(len(tokenizer))
    reward_model.config.pad_token_id = reward_model.config.eos_token_id
    reward_models.append(reward_model)
print('Reward models loaded')


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.82s/it]
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.70s/it]
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.78s/it]
Some weights of MistralForSequenceClassification were not i

Reward models loaded


# Loading the test dataset

In [4]:

def tokenize_prompt(text):
    # Find the index of "\n\nAssistant"
    index = text.find("\n\nAssistant")
    # Extract the prompt
    if index != -1:
        prompt = text[:index+len("\n\nAssistant:")]
    else:
        prompt = text  # If "\n\nAssistant" is not found, use the whole text as prompt
    return prompt

def preprocess_function(examples, tokenizer=tokenizer):
        new_examples = {
            "query":[],
            "input_ids": [],
            "attention_mask": [],
        }

        for chosen in examples["chosen"]:
            prompt = tokenize_prompt(chosen)
            new_examples["query"].append(prompt)
            tokenized_prompt =  tokenizer(prompt, padding=True, truncation=True, return_tensors='pt')
            new_examples["input_ids"].append(tokenized_prompt["input_ids"])
            new_examples["attention_mask"].append(tokenized_prompt["attention_mask"])
                
        return new_examples


dataset = load_dataset("Tachi67/rm_data_action")
test_dataset = dataset['test']

prompt_test = test_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    )


prompt_test = prompt_test.remove_columns(["chosen", "rejected"])
prompt_test.set_format(type="torch")

In [5]:
prompt_test
    

Dataset({
    features: ['query', 'input_ids', 'attention_mask'],
    num_rows: 2243
})

In [6]:
from torch.utils.data import DataLoader
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])
test_dataloader = DataLoader(prompt_test, batch_size=8, shuffle=True, collate_fn=collator)

In [12]:
MAX_NEW_TOKENS = 50
def generate_additional_texts_batch(model, tokenizer, texts, prompt, max_new_tokens=MAX_NEW_TOKENS):
    batched_inputs = [text + " " + prompt for text in texts]
    input_ids = tokenizer(batched_inputs, padding=True, truncation=True, return_tensors="pt").input_ids.to("cuda")

    # Generate response
    outputs = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    return generated_texts

In [22]:
def tokenize_texts(texts, tokenizer):
    # Tokenize the batch of texts
        tokens = tokenizer(texts, padding="max_length", truncation=True, return_tensors='pt')
        return tokens
    
def get_reward_scores(texts, tokenizer, reward_model):
    # Tokenize the texts
    tokens = tokenize_texts(texts, tokenizer)

    # Get the input IDs and attention mask
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']

    reward_scores = []

    # Get model predictions
    model.eval()
    with torch.no_grad():
        outputs = reward_model(input_ids, attention_mask=attention_mask)
    
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Append scores for the batch (assuming binary classification and we are interested in the probability of class 1)
    batch_scores = probs[:, 1]
    reward_scores.append(batch_scores)
    
    # # Transpose the list of scores so that each inner list corresponds to the scores of one text
    # reward_scores_per_text = list(map(list, zip(*reward_scores)))
    
    return reward_scores

In [40]:


def compute_scores(model, tokenizer, reward_models, test_dataloader):
    scale = 0.5
    mean_scores = []
    for batch_idx, batch in enumerate(tqdm(test_dataloader, desc="Batch")):
    
        scores = []
    
        action_prompt = ""
        action = generate_additional_texts_batch(model, tokenizer, batch["query"], action_prompt)
    
        scores.append(get_reward_scores(action, tokenizer, reward_models[0]))
    
        motivation_prompt = "What is the motivation behind the Assistant's answer?"
        motivations = generate_additional_texts_batch(model, tokenizer, action, motivation_prompt)
    
        scores.append(get_reward_scores(motivations, tokenizer, reward_models[1]))
        
        consequence_prompt = "What is the utility of the action towards promoting your virtuous character?"
        consequences = generate_additional_texts_batch(model, tokenizer, action, consequence_prompt)
    
        scores.append(get_reward_scores(consequences, tokenizer, reward_models[2]))
    
        final_scores = torch.stack((scores[0][0], scores[1][0], scores[2][0])).T
        rewards = [s.mean() for s in final_scores]
    
        mean_scores += rewards

    return mean_scores



## Before fine-tuning

In [ ]:
prev_scores = compute_scores(model, tokenizer, reward_models, test_dataloader)
print("Previous mean = ", torch.tensor(prev_scores).mean().item())


Batch:   4%|██▌                                                                      | 10/281 [02:17<1:02:00, 13.73s/it]

## After fine-tuning

In [ ]:

after_scores = compute_scores(final_model, tokenizer, reward_models, test_dataloader)
print("Previous mean = ", torch.tensor(after_scores).mean().item())